In [1]:
################################################################
## GPU Should be enabled in colab
## This notebook is reproducing results from https://github.com/kanekomasahiro/gp_debias
## You should upload debiasing.ipynb into https://colab.research.google.com/ to run it
################################################################
!git clone https://github.com/pavelbystrov1/debiasing_evaluation
!git clone https://github.com/kanekomasahiro/gp_debias
!cd ./gp_debias; mkdir d; cd d; git clone https://github.com/uclanlp/gn_glove.git
!paste ./gp_debias/d/gn_glove/wordlist/female_word_file.txt ./gp_debias/d/gn_glove/wordlist/male_word_file.txt > ./gp_debias/wordlist/gender_pair.tsv 
!cp -r ./gp_debias/d/gn_glove/SemBias ./gp_debias/
!ls ./gp_debias
##############################################################################
# Glove embedding trained on wikidump dataset can be downloaded using code below or manually - https://drive.google.com/open?id=1jrbQmpB5ZNH4w54yujeAvNFAfVEG0SuE
# GN-Glove embedding trained on wikidump dataset can be downloaded using code below or manually - https://drive.google.com/file/d/1v82WF43w-lE-vpZd0JC1K8WYZQkTy_ii/
##############################################################################
!gdown --id '1jrbQmpB5ZNH4w54yujeAvNFAfVEG0SuE'
!gdown --id '1v82WF43w-lE-vpZd0JC1K8WYZQkTy_ii'
##########################################
# unzip GloVe embedding
##########################################
!cd gp_debias; mkdir -p src/glove_model
!cp GloVe.zip ./gp_debias/src/glove_model/
!cd gp_debias/src/glove_model; unzip GloVe.zip
!cd gp_debias/src/glove_model; sed -i "1s/^/322636 300\n/" vectors.txt
!cd gp_debias/; mkdir embeddings
!mv gp_debias/src/glove_model/vectors.txt gp_debias/embeddings/glove.txt
!cd gp_debias; mkdir -p src/debiased_glove
!ls gp_debias/embeddings/

Cloning into 'debiasing_evaluation'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.
Cloning into 'gp_debias'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Unpacking objects: 100% (66/66), done.
Cloning into 'gn_glove'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 199 (delta 2), reused 6 (delta 2), pack-reused 193
Receiving objects: 100% (199/199), 67.78 KiB | 4.84 MiB/s, done.
Resolving deltas: 100% (88/88), done.
d	     eval_word_embeddings.py  LICENSE	 run.sh   src
download.sh  hyperparams	      README.md  SemBias  wordlist
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1

In [2]:
###########################################
# create classifiers for glove
# save gp_glove embedding
###########################################
!cd gp_debias; python src/train.py glove
!cd gp_debias; python src/eval.py glove
!ls gp_debias/src/debiased_glove
#############################################################
# create folder for evaluation of generated word embeddings
# and copy glove.txt and gp_glove.txt there
#############################################################
!cd ./debiasing_evaluation; mkdir glove
!mv gp_debias/embeddings/glove.txt debiasing_evaluation/glove/glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.txt debiasing_evaluation/glove/gp_glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.bin debiasing_evaluation/glove/gp_glove.bin
#######################################
#      prepare data for GN-Glove
#######################################
!cd gp_debias; 
!cp /content/GN-GloVe-L1-0.8-0.8.txt.zip /content/gp_debias/src/glove_model/
!cd gp_debias/src/glove_model; unzip "GN-GloVe-L1-0.8-0.8.txt.zip"
!cd gp_debias/src/glove_model; sed -i "1s/^/322636 300\n/" vectors300.txt
!cp gp_debias/src/glove_model/vectors300.txt gp_debias/src/glove_model/glove.txt
!mv gp_debias/src/glove_model/glove.txt gp_debias/embeddings/glove.txt
!ls gp_debias/src/glove_model

Loading word embedding
Loading data
Pre-training autoencoder
/content/gp_debias/src/pre_train_autoencoder.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  eval_inputs = torch.split(torch.FloatTensor(emb_list[:hp.pta_dev_num]), hp.pta_batch_size)
src/train.py:426: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  train_female_embs = [encoder(torch.FloatTensor(emb[word[0]]).cuda()).data if hp.gpu >= 0 else encod

In [3]:
###########################################
# create classifiers for GN-glove and save gp_gn_glove embedding
###########################################
!cd gp_debias; python src/train.py glove
!cd gp_debias; python src/eval.py glove
!ls gp_debias/src/debiased_glove
#############################################################
# copy GN-glove and gp_gn_glove there
#############################################################
!mv gp_debias/embeddings/glove.txt debiasing_evaluation/glove/gn_glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.txt debiasing_evaluation/glove/gp_gn_glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.bin debiasing_evaluation/glove/gp_gn_glove.bin
!ls debiasing_evaluation/glove/


Loading word embedding
Loading data
Pre-training autoencoder
/content/gp_debias/src/pre_train_autoencoder.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  eval_inputs = torch.split(torch.FloatTensor(emb_list[:hp.pta_dev_num]), hp.pta_batch_size)
src/train.py:426: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  train_female_embs = [encoder(torch.FloatTensor(emb[word[0]]).cuda()).data if hp.gpu >= 0 else encod

In [4]:
###############################################################
# Load SimLex, datasets from Semantic-measure-assessment study and Google anthologies
# Copy wordlists, SemBias and other test datasets to do evaluation later
###############################################################
!wget https://fh295.github.io/SimLex-999.zip
!git clone https://github.com/MohamedAliHadjTaieb/Semantic-measure-assessment-review-study
!wget http://download.tensorflow.org/data/questions-words.txt
!unzip SimLex-999.zip
!mv questions-words.txt debiasing_evaluation/glove/google_anthology_ds.txt
!mv SimLex-999/SimLex-999.txt debiasing_evaluation/glove/SimLex-999.txt
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Relatedness/MT771.csv" debiasing_evaluation/glove/MTurk.csv
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Relatedness/RW2034.csv" debiasing_evaluation/glove/
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Relatedness/WordSim353.csv" debiasing_evaluation/glove/
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Similarity/RG65.csv" debiasing_evaluation/glove/
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Similarity/MEN3000.csv" debiasing_evaluation/glove/
!cp -r ./gp_debias/d/gn_glove/SemBias debiasing_evaluation/glove/
!cp -r ./gp_debias/wordlist debiasing_evaluation/glove/
############################################
#            Preparation DONE
############################################

--2022-04-29 16:57:53--  https://fh295.github.io/SimLex-999.zip
Resolving fh295.github.io (fh295.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to fh295.github.io (fh295.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16805 (16K) [application/zip]
Saving to: ‘SimLex-999.zip’

SimLex-999.zip      100%[===================>]  16.41K  --.-KB/s    in 0s      

2022-04-29 16:57:54 (32.7 MB/s) - ‘SimLex-999.zip’ saved [16805/16805]

Cloning into 'Semantic-measure-assessment-review-study'...
remote: Enumerating objects: 1365, done.
remote: Total 1365 (delta 0), reused 0 (delta 0), pack-reused 1365
Receiving objects: 100% (1365/1365), 690.99 KiB | 2.42 MiB/s, done.
Resolving deltas: 100% (618/618), done.
--2022-04-29 16:57:55--  http://download.tensorflow.org/data/questions-words.txt
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.76.128, 2a00:1450:400c:c00::80
Connecting to download.ten

In [5]:
##############################################
# for Google colab path starts with /content/ if you will run this ipynb-file localy please remove "/content" from directories below
##############################################
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


from numpy import dot
from numpy.linalg import norm
from gensim.models import KeyedVectors
import pathlib
from pathlib import Path

import numpy as np
from scipy import stats
from functools import reduce
import pandas as pd
import seaborn as sns

SEM_BIAS = '/content/debiasing_evaluation/glove/SemBias/SemBias'
GP_GN = '/content/debiasing_evaluation/glove/gp_gn_glove.txt'
GN_GLOVE = '/content/debiasing_evaluation/glove/gn_glove.txt'
GLOVE = '/content/debiasing_evaluation/glove/glove.txt'
GOOGLE_ANT = '/content/debiasing_evaluation/glove/google_anthology_ds.txt'
RG65 = '/content/debiasing_evaluation/glove/RG65.csv'
WS = '/content/debiasing_evaluation/glove/WordSim353.csv'
MTURK = '/content/debiasing_evaluation/glove/MTurk.csv'
RW = '/content/debiasing_evaluation/glove/RW2034.csv'
MEN = '/content/debiasing_evaluation/glove/MEN3000.csv'
SIMLEX = '/content/debiasing_evaluation/glove/SimLex-999.txt'
SEM_EVAL = '/content/debiasing_evaluation/glove/SemEval'
GP_GLOVE = '/content/debiasing_evaluation/glove/gp_glove.txt'
STEREOTYPE = '/content/debiasing_evaluation/glove/wordlist/stereotype_list.tsv'
NOGENDER = '/content/debiasing_evaluation/glove/wordlist/no_gender_list.tsv'
GENDER = '/content/debiasing_evaluation/glove/wordlist/gender_pair.tsv'


In [6]:
emb_gn = KeyedVectors.load_word2vec_format(GP_GLOVE, binary=False)

In [7]:
def get_vocabulary(fn = GLOVE):
    with open(fn, "r") as f:
        vocab = [line.lower().split()[0] for line in f]
    return vocab[1:]


def get_analogy_ds(fn = GOOGLE_ANT):
    with open(fn, "r") as f:
        vocab = [line.lower().split() for line in f if line[0] != ':']
    return vocab


def get_semeval(path = SEM_EVAL):
    fnames = Path(path).rglob("*.txt")
    full_lst = []
    for n in fnames:
        pth = str(n)
        with open(pth, "r") as f:
            vocab = [line.replace('"', "").lower().split() for line in f if line[0] != '#']
        full_lst = full_lst + vocab
    res = [[p[0].split(':')] + [p[1].split(':')]+[p[2].split(':')] + [p[3].split(':')] for p in full_lst]
    return res

def cosine_sim(v, w):
    """cosine similarity"""
    return np.dot(w, v)/((np.linalg.norm(w)) * (np.linalg.norm(v) + 0.000001))


def find_simi(lst, a, b, c, vocab, emb):
    """find argmax using 3COSADD method"""
    v1 = b - a + c
    amax = 0.
    result = ""
    for w2 in vocab:
        if w2 not in lst:
            sim = cosine_sim(emb[w2], v1)
            if sim > amax:
                result = w2
                amax = sim
    return result, amax


def cosine_sim(v, w, eps = 0.001):
    """cosine similarity"""
    return np.dot(w, v)/((np.linalg.norm(w)+0.000001) * (np.linalg.norm(v)+eps))


def find_simi(lst, a, b, c, vocab, emb):
    """find argmax using 3COSADD method"""
    v1 = b - a + c
    amax = 0.
    result = ""
    for w2 in vocab:
        if w2 not in lst:
            sim = cosine_sim(emb[w2], v1)
            if sim > amax:
                result = w2
                amax = sim
    return result, amax


def calculate_acc_for_analogies(anthologies, emb_gn, vocab_gn):
    """ check overall accuracy of anthologies """
    corrects = 0
    total = 0
    a, b, c = "", "", ""
    emb1, emb2, emb3 = [0], [0], [0]
    for tup in anthologies:
        e1 = emb_gn[tup[0]] if a != tup[0] else emb1
        e2 = emb_gn[tup[1]] if b != tup[1] else emb2
        e3 = emb_gn[tup[2]] if c != tup[2] else emb3
        word, sim = find_simi(tup[:3], e1, e2, e3, vocab_gn, emb_gn)
        if word == tup[3]:
            corrects +=1
        total +=1
        emb1, emb2, emb3 = e1, e2, e3
        a, b, c = tup[0], tup[1], tup[2]
        if total % 50 == 0:
            print(total, *tup, word, sim, "actual:", tup[3], corrects/total)
    return corrects, total


analogies = get_analogy_ds()
vocab_gp = get_vocabulary(fn = GP_GLOVE)

In [ ]:
##################################################################
# It might take up to 48 hours to reproduce Google anthology tests !!
# you can run 2 next cells to verify results for 67 and 400 rows
#####################################################################

print(f"GN GLOVE check")
corrects, total = calculate_acc_for_analogies(analogies, emb_gn, vocab_gp)
print(f"Final score: {corrects/total} {corrects} {total}")

GN GLOVE check
50 bangkok thailand hanoi vietnam vietnam 0.6850367252897471 actual: vietnam 0.96
100 berlin germany oslo norway norway 0.7555886964501195 actual: norway 0.97
150 cairo egypt baghdad iraq iraq 0.7404456221084577 actual: iraq 0.96
200 havana cuba islamabad pakistan pakistan 0.7136669378448934 actual: pakistan 0.955
250 islamabad pakistan rome italy italy 0.7133663294290572 actual: italy 0.956
300 london england berlin germany germany 0.6600025102687568 actual: germany 0.96
350 moscow russia kabul afghanistan afghanistan 0.7697258532627865 actual: afghanistan 0.9628571428571429
400 paris france tokyo japan japan 0.7510880087797088 actual: japan 0.9625
450 stockholm sweden canberra australia australia 0.6283671971970209 actual: australia 0.9644444444444444
500 tokyo japan moscow russia russia 0.804189515815529 actual: russia 0.964
550 accra ghana apia samoa samoa 0.587528527154653 actual: samoa 0.9636363636363636
600 algiers algeria belgrade serbia serbia 0.7395836943844512

4450 tehran iran tunis tunisia tunisia 0.7079895477371879 actual: tunisia 0.9539325842696629
4500 thimphu bhutan algiers algeria algeria 0.6406752017206051 actual: algeria 0.9544444444444444
4550 tirana albania bangkok thailand thailand 0.7156899977305886 actual: thailand 0.9542857142857143
4600 tokyo japan budapest hungary hungary 0.7537721199125148 actual: hungary 0.9547826086956521
4650 tunis tunisia abuja nigeria nigeria 0.6735391874116704 actual: nigeria 0.9550537634408602
4700 vaduz liechtenstein baku azerbaijan azerbaijan 0.6650787422164804 actual: azerbaijan 0.955531914893617
4750 valletta malta brussels belgium belgium 0.6951510295571067 actual: belgium 0.9555789473684211
4800 vientiane laos yerevan armenia armenia 0.5864594123055564 actual: armenia 0.9558333333333333
4850 vilnius lithuania athens greece greece 0.7761045093552832 actual: greece 0.9562886597938144
4900 warsaw poland bishkek kyrgyzstan kyrgyzstan 0.6503349714256641 actual: kyrgyzstan 0.9567346938775511
4950 wind

8650 man woman nephew niece niece 0.7793577951133991 actual: niece 0.8048554913294798
8700 policeman policewoman uncle aunt grandfather 0.547061432097489 actual: aunt 0.8050574712643678
8750 son daughter grandson granddaughter granddaughter 0.8967902690788242 actual: granddaughter 0.8054857142857142
8800 stepbrother stepsister policeman policewoman undercover 0.43563202696466263 actual: policewoman 0.8060227272727273
8850 uncle aunt brothers sisters sisters 0.7337796051554695 actual: sisters 0.8063276836158192
8900 amazing amazingly usual usually fairly 0.4353555396154974 actual: usually 0.803932584269663
8950 calm calmly rapid rapidly industrialization 0.43402225290446295 actual: rapidly 0.8005586592178771
9000 complete completely infrequent infrequently totally 0.44946878086069386 actual: infrequently 0.7973333333333333
9050 efficient efficiently usual usually occasionally 0.4133883456542893 actual: usually 0.794696132596685
9100 free freely rapid rapidly rapidly 0.48413528818485485 

12800 strange strangest tall tallest tallest 0.5075045533868683 actual: tallest 0.706328125
12850 strong strongest hot hottest billboard 0.516695918773027 actual: hottest 0.7059922178988327
12900 tall tallest weird weirdest rarest 0.370394211672587 actual: weirdest 0.7055038759689922
12950 tasty tastiest lucky luckiest igbinedion 0.41092592835639336 actual: luckiest 0.7044015444015445
13000 weak weakest big biggest biggest 0.48262939118394593 actual: biggest 0.7028461538461539
13050 weird weirdest simple simplest simplest 0.43059809798332216 actual: simplest 0.7022988505747126
13100 young youngest dark darkest darker 0.4597670122949482 actual: darkest 0.7012977099236641
13150 code coding see seeing perceptual 0.36208053735609247 actual: seeing 0.6999239543726236
13200 debug debugging increase increasing increased 0.61995257558958 actual: increasing 0.6991666666666667
13250 decrease decreasing vanish vanishing vanishes 0.5576561961973672 actual: vanishing 0.6988679245283019
13300 discov

17000 slowing slowed dancing danced dance 0.6332133366558019 actual: danced 0.7054117647058824
17050 spending spent increasing increased increased 0.615547068293956 actual: increased 0.7051026392961877
17100 striking struck screaming screamed screams 0.4855845047339877 actual: screamed 0.7047953216374269
17150 taking took walking walked walk 0.6018770967001634 actual: walked 0.7041399416909621
17200 thinking thought hitting hit hit 0.6038662563281537 actual: hit 0.7038953488372093
17250 vanishing vanished running ran ran 0.5933227972472171 actual: ran 0.7036521739130435
17300 walking walked swimming swam swam 0.5411557464052763 actual: swam 0.7033526011560693
17350 banana bananas color colors colour 0.6522338226554668 actual: colors 0.7027089337175793
17400 bird birds machine machines machines 0.6754873897140109 actual: machines 0.7032183908045977
17450 bottle bottles bird birds birds 0.640972884498438 actual: birds 0.7033810888252149
17500 car cars finger fingers fingers 0.65270655923

In [8]:
# Test for semantic analogies using previous results printed above
corrects, total = 13710, 19544

def calculate_acc_for_sem_analogies(analogies, emb_gn, vocab_gn):
    """ check accuracy of sem analogies
        initial accuracy was taken from row '8800 ...' of previous cell
    """
    previous_res = "8800 stepbrother stepsister policeman policewoman undercover 0.43563202696466263 actual: policewoman 0.8060227272727273"
    print(previous_res)
    total = 8800
    corrects = int(0.8060227272727273 * total)
    a, b, c = "", "", ""
    emb1, emb2, emb3 = [0], [0], [0]
    for tup in analogies[8800:8867]:
        # saving vectors to speed up computation
        e1 = emb_gn[tup[0]] if a != tup[0] else emb1
        e2 = emb_gn[tup[1]] if b != tup[1] else emb2
        e3 = emb_gn[tup[2]] if c != tup[2] else emb3
        # trying to find similar word as argmax(3COSADD(e2-e1+e3, E(w))) for all words w in vocab except tup[:3]
        word, sim = find_simi(tup[:3], e1, e2, e3, vocab_gn, emb_gn)
        if word == tup[3]:
            corrects +=1
        total +=1
        emb1, emb2, emb3 = e1, e2, e3
        a, b, c = tup[0], tup[1], tup[2]
        if total % 200 == 0:
            print(total, *tup, word, sim, "actual:", tup[3], corrects/total)
    print(total, *tup, word, sim, "actual:", tup[3], corrects/total)
    return corrects , total

print(f"GP-GLOVE check")
sem_corrects, sem_total = calculate_acc_for_sem_analogies(analogies, emb_gn, vocab_gp)
print(f"GP-GLOVE accuracy of sem analogies: {sem_corrects/sem_total} {sem_corrects} {sem_total}")
syn_acc = (corrects - sem_corrects) / (total - sem_total)
print(f"GP-GLOVE accuracy of syn analogies: {syn_acc} correct answers: {corrects - sem_corrects}")
print(f"GP-GLOVE overal accuracy for analogies: {corrects / total} ")

GP-GLOVE check
8800 stepbrother stepsister policeman policewoman undercover 0.43563202696466263 actual: policewoman 0.8060227272727273
8867 uncle aunt stepbrother stepsister stepmother 0.529885949010886 actual: stepsister 0.8065862185632119
GP-GLOVE accuracy of sem analogies: 0.8065862185632119 7152 8867
GP-GLOVE accuracy of syn analogies: 0.6142174768193313 correct answers: 6558
GP-GLOVE overal accuracy for analogies: 0.7014940646745804 


In [ ]:
# Test for semantic analogies using previous results printed above
def calculate_acc_for_400(analogies, emb_gn, vocab_gn):
    """ check accuracy of sem analogies
        initial accuracy was taken from row '8800 ...' of previous cell
    """
    total = 0
    corrects = 0
    a, b, c = "", "", ""
    emb1, emb2, emb3 = [0], [0], [0]
    for tup in analogies[:400]:
        # saving vectors to speed up computation
        e1 = emb_gn[tup[0]] if a != tup[0] else emb1
        e2 = emb_gn[tup[1]] if b != tup[1] else emb2
        e3 = emb_gn[tup[2]] if c != tup[2] else emb3
        # trying to find similar word as argmax(3COSADD(e2-e1+e3, E(w))) for all words w in vocab except tup[:3]
        word, sim = find_simi(tup[:3], e1, e2, e3, vocab_gn, emb_gn)
        if word == tup[3]:
            corrects +=1
        total +=1
        emb1, emb2, emb3 = e1, e2, e3
        a, b, c = tup[0], tup[1], tup[2]
        if total % 200 == 0:
            print(total, *tup, word, sim, "actual:", tup[3], corrects/total)
    print(total, *tup, word, sim, "actual:", tup[3], corrects/total)
    return corrects , total

sem_corrects, sem_total = calculate_acc_for_400(analogies, emb_gn, vocab_gp)
print(f"GP-GLOVE accuracy of 400 sem analogies: {sem_corrects/sem_total} {sem_corrects} {sem_total}")